# Student Loan Risk with Deep Learning

In [82]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [83]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [84]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [85]:
# Get information about the DataFrame
loans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   payment_history         1599 non-null   float64
 1   location_parameter      1599 non-null   float64
 2   stem_degree_score       1599 non-null   float64
 3   gpa_ranking             1599 non-null   float64
 4   alumni_success          1599 non-null   float64
 5   study_major_code        1599 non-null   float64
 6   time_to_completion      1599 non-null   float64
 7   finance_workshop_score  1599 non-null   float64
 8   cohort_ranking          1599 non-null   float64
 9   total_loan_score        1599 non-null   float64
 10  financial_aid_score     1599 non-null   float64
 11  credit_ranking          1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [86]:
# Check for missing values
loans_df.isnull().sum()

payment_history           0
location_parameter        0
stem_degree_score         0
gpa_ranking               0
alumni_success            0
study_major_code          0
time_to_completion        0
finance_workshop_score    0
cohort_ranking            0
total_loan_score          0
financial_aid_score       0
credit_ranking            0
dtype: int64

In [87]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [88]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display a sample of y
y.head()


0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [89]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop('credit_ranking', axis=1)

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [90]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)



### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [91]:
# Create a StandardScaler instance
scaler = StandardScaler()


# Fit the scaler to the features training dataset
X_train_scaled = scaler.fit_transform(X_train)


# Fit the scaler to the features training dataset
X_test_scaled = scaler.transform(X_test)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [92]:
# Define the the number of inputs (features) to the model
dim_input = X_train_scaled.shape[1]

# Review the number of features
print(f'Feature Count: {dim_input}')

Feature Count: 11


In [93]:
# Define the number of hidden nodes for the first hidden layer
first_hidden_node_count = 6

# Define the number of hidden nodes for the second hidden layer
second_hidden_node_count = 3

# Define the number of neurons in the output layer
output_node_count = 1

In [94]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(units=first_hidden_node_count, activation="relu", input_dim = dim_input))

# Add the second hidden layer
model.add(Dense(units=second_hidden_node_count, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=output_node_count, activation="sigmoid"))


c:\Users\lonni\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [95]:
# Display the Sequential model summary

model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 6)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 3)              │            21 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97 (388.00 B)

 Trainable params: 97 (388.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [96]:
# Compile the Sequential model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [97]:
# Fit the model using 50 epochs and the training data
model.fit(X_train_scaled, y_train, epochs=50)


Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4722 - loss: 0.7116 
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5215 - loss: 0.6986 
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5416 - loss: 0.6924 
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5511 - loss: 0.6858 
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5889 - loss: 0.6756 
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5841 - loss: 0.6656 
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5825 - loss: 0.6523 
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6076 - loss: 0.6237
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6526 - loss: 0.6117 
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6641 - loss: 0.6035 
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7140 - loss: 0.5732 
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [98]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test)

# Display the model loss and accuracy results
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7423 - loss: 0.4991  
Loss: 0.5281
Accuracy: 0.7275


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [99]:
# Set the model's file path
path = 'student_loans.keras'

# Export your model to a keras file
model.save(path)


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [100]:
# Set the model's file path
new_path = 'student_loans.keras'

# Load the model to a new object
new_model = tf.keras.models.load_model(new_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [101]:
# Make predictions with the test data
test_predictions = new_model.predict(X_test_scaled)


# Display a sample of the predictions
test_predictions[:5]

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


array([[0.48783466],
       [0.25680324],
       [0.81096464],
       [0.6941798 ],
       [0.9795653 ]], dtype=float32)

In [102]:
# Save the predictions to a DataFrame and round the predictions to binary results
prediction_results = pd.DataFrame((test_predictions > 0.5).astype('uint8'), columns=["predictions"])
prediction_results

,predictions
0,0
1,0
2,1
3,1
4,1
...,...
395,1
396,0
397,1
398,0


### Step 4: Display a classification report with the y test data and predictions

In [103]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, prediction_results))


              precision    recall  f1-score   support

           0       0.69      0.77      0.73       188
           1       0.77      0.69      0.73       212

    accuracy                           0.73       400
   macro avg       0.73      0.73      0.73       400
weighted avg       0.73      0.73      0.73       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

*I would need to collect:*

- Education: Major, Grades, Simesters remaining until graduation
    - shows possible future earnings and repayment timeline    
- Finances: Income, Credit score, Financial aid
    - helps to determine what type of loan should be offered
- Loan Details: Rates, Amount needed, Loan duration, Loan type
    - helps to determine what type of loan is needed
- Similar students: Compare against similar student's repayment behaviors
    - base recomendations off of real world outcomes of similar students

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

*I used contenet-based filtering*
- Most of the data I used (such as education and financial status) consists of objective information about the student. I am recommending loans that are tailored to the student’s specific attributes and needs. This approach allows for personalized recommendations based on their individual profile.

I have selected mostly content based filtering utilizing objective information about the applicant. I would leverage some collaborative filtering in order to recommend loans that have worked for similar students.  

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

*Two challenges that I would have to consider is privacy and government regulation.*
 - Privacy: I would be handling personal sensitive information. Students may not want to share the details needed to make a good recomendation. 
 - Regulation: finance is a heavily regulated industry. I would have to meet these regulations in order to produce fair and unbiased recommendations as well as avoid regulatory penalties.